**Name**: Anugya Shaw <br>
**Roll no**: D19006

Create a feed forward network using MNIST Classification dataset and experiment on its parameters using different optimizers to achieve best accuracy with minimum and maximum number of parameters.

### Imporing necessary libraries

In [40]:
import torch
from torch.utils.data import TensorDataset ,DataLoader
from torch import nn,optim
import torch.nn.functional as F

In [41]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

### Reading the train and test datasets

In [44]:
# Importing Training and Test data
train=pd.read_csv('C:/Users/Anugya/Downloads/digit-recognizer/train.csv')
test=pd.read_csv('C:/Users/Anugya/Downloads/digit-recognizer/test.csv')
print('Training Data shape: ',train.shape)
print('Test Data shape: ',test.shape)

Training Data shape:  (42000, 785)
Test Data shape:  (28000, 784)


In [45]:
x=train.drop("label",axis=1)
y=np.array(train['label'])

In [133]:
# Convert Data to Tensor
torch_X_train = torch.from_numpy(x.values).type(torch.FloatTensor)/255
torch_y_train = torch.from_numpy(y).type(torch.LongTensor)

myDataset = torch.utils.data.TensorDataset(torch_X_train,torch_y_train)
valid_no  = int(0.2 * len(myDataset))

# Divide the data into trainset and testset
trainSet,testSet = torch.utils.data.random_split(myDataset,(len(myDataset)-valid_no,valid_no))
print(f"len of Train-Set {len(trainSet)} , len of Test-Set {len(testSet)}")


# Load Data in memory
train_loader  = DataLoader(trainSet , batch_size=64 ,shuffle=True) 
test_loader  = DataLoader(testSet , batch_size=64 ,shuffle=True)

len of Train-Set 33600 , len of Test-Set 8400


### Building the Neural Network

In [130]:
from torch import nn, optim
class Network(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(784, 25)
        self.fc2 = nn.Linear(25, 25)
        self.fc3 = nn.Linear(25, 25)
        self.fc4 = nn.Linear(25, 25)
        self.fc5 = nn.Linear(25, 10)

        # Dropout module with drop probability
        self.dropout = nn.Dropout(p=0.2)

    def forward(self, x):
        # make sure input tensor is flattened
        x = x.view(x.shape[0], -1)

        x = self.dropout(F.relu(self.fc1(x)))
        x = self.dropout(F.relu(self.fc2(x)))
        x = self.dropout(F.relu(self.fc3(x)))
        x = self.dropout(F.relu(self.fc4(x)))

        # No dropout at output
        x = F.log_softmax(self.fc5(x), dim=1)

        return x
        
model=Network()
print(model)
# specify loss function
criterion = nn.CrossEntropyLoss()

# specify optimizer
optimizer = optim.SGD(model.parameters(),lr=0.01, momentum=0.9,nesterov=True)
#optimizer = optim.SGD(model.parameters(),lr=0.2, momentum=0.9,weight_decay=1e-68nesterov=True)
#optimizer = optim.Adam(model.parameters(),lr=0.001)
                      

Network(
  (fc1): Linear(in_features=784, out_features=25, bias=True)
  (fc2): Linear(in_features=25, out_features=25, bias=True)
  (fc3): Linear(in_features=25, out_features=25, bias=True)
  (fc4): Linear(in_features=25, out_features=25, bias=True)
  (fc5): Linear(in_features=25, out_features=10, bias=True)
  (dropout): Dropout(p=0.2)
)


### Training the Model

In [131]:
epochs=10
train_losses,test_losses=[],[]
for e in range(epochs):
    running_loss=0
    for images,labels in train_loader:
        optimizer.zero_grad()
        log_ps=model(images)
        loss=criterion(log_ps,labels)
        loss.backward()
        optimizer.step()
        running_loss+=loss.item()
        
    else:
        test_loss=0
        accuracy=0
        
        with torch.no_grad():
            model.eval()
            for images,labels in test_loader:
                log_ps=model(images)
                test_loss+=criterion(log_ps,labels)
                ps=torch.exp(log_ps)
                top_p,top_class=ps.topk(1,dim=1)
                equals=top_class==labels.view(*top_class.shape)
                accuracy+=torch.mean(equals.type(torch.FloatTensor))
        model.train()
        train_losses.append(running_loss/len(train_loader))
        test_losses.append(test_loss/len(test_loader))

        print("Epoch: {}/{}.. ".format(e+1, epochs),
              "Training Loss: {:.3f}.. ".format(running_loss/len(train_loader)),
              "Test Loss: {:.3f}.. ".format(test_loss/len(test_loader)),
              "Test Accuracy: {:.3f}".format(accuracy/len(test_loader)))

Epoch: 1/10..  Training Loss: 1.772..  Test Loss: 0.768..  Test Accuracy: 0.813
Epoch: 2/10..  Training Loss: 0.843..  Test Loss: 0.435..  Test Accuracy: 0.892
Epoch: 3/10..  Training Loss: 0.655..  Test Loss: 0.370..  Test Accuracy: 0.911
Epoch: 4/10..  Training Loss: 0.568..  Test Loss: 0.325..  Test Accuracy: 0.914
Epoch: 5/10..  Training Loss: 0.517..  Test Loss: 0.291..  Test Accuracy: 0.922
Epoch: 6/10..  Training Loss: 0.493..  Test Loss: 0.285..  Test Accuracy: 0.924
Epoch: 7/10..  Training Loss: 0.466..  Test Loss: 0.267..  Test Accuracy: 0.927
Epoch: 8/10..  Training Loss: 0.445..  Test Loss: 0.272..  Test Accuracy: 0.927
Epoch: 9/10..  Training Loss: 0.433..  Test Loss: 0.245..  Test Accuracy: 0.933
Epoch: 10/10..  Training Loss: 0.424..  Test Loss: 0.260..  Test Accuracy: 0.928


### Experimentation details by changing hyperparatemeters

**1. Optimizer: SGD, learning rate = 0.002, momentum = 0.9, nesterov = True**

In [143]:
from torch import nn, optim
class Network(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(784, 240)
        self.fc2 = nn.Linear(240, 200)
        self.fc3 = nn.Linear(200, 160)
        self.fc4 = nn.Linear(160, 120)
        self.fc5 = nn.Linear(120, 10)

        # Dropout module with drop probability
        self.dropout = nn.Dropout(p=0.2)

    def forward(self, x):
        # make sure input tensor is flattened
        x = x.view(x.shape[0], -1)

        x = self.dropout(F.relu(self.fc1(x)))
        x = self.dropout(F.relu(self.fc2(x)))
        x = self.dropout(F.relu(self.fc3(x)))
        x = self.dropout(F.relu(self.fc4(x)))

        # No dropout at output
        x = F.log_softmax(self.fc5(x), dim=1)

        return x
        
model=Network()
print(model)
# specify loss function
criterion = nn.CrossEntropyLoss()

# specify optimizer
optimizer = optim.SGD(model.parameters(),lr=0.002, momentum=0.9,nesterov=True)


Network(
  (fc1): Linear(in_features=784, out_features=240, bias=True)
  (fc2): Linear(in_features=240, out_features=200, bias=True)
  (fc3): Linear(in_features=200, out_features=160, bias=True)
  (fc4): Linear(in_features=160, out_features=120, bias=True)
  (fc5): Linear(in_features=120, out_features=10, bias=True)
  (dropout): Dropout(p=0.2)
)


In [145]:
epochs=10
train_losses,test_losses=[],[]
for e in range(epochs):
    running_loss=0
    for images,labels in train_loader:
        optimizer.zero_grad()
        log_ps=model(images)
        loss=criterion(log_ps,labels)
        loss.backward()
        optimizer.step()
        running_loss+=loss.item()
        
    else:
        test_loss=0
        accuracy=0
        
        with torch.no_grad():
            model.eval()
            for images,labels in test_loader:
                log_ps=model(images)
                test_loss+=criterion(log_ps,labels)
                ps=torch.exp(log_ps)
                top_p,top_class=ps.topk(1,dim=1)
                equals=top_class==labels.view(*top_class.shape)
                accuracy+=torch.mean(equals.type(torch.FloatTensor))
        model.train()
        train_losses.append(running_loss/len(train_loader))
        test_losses.append(test_loss/len(test_loader))

        print("Epoch: {}/{}.. ".format(e+1, epochs),
              "Training Loss: {:.3f}.. ".format(running_loss/len(train_loader)),
              "Test Loss: {:.3f}.. ".format(test_loss/len(test_loader)),
              "Test Accuracy: {:.3f}".format(accuracy/len(test_loader)))

Epoch: 1/10..  Training Loss: 0.195..  Test Loss: 0.150..  Test Accuracy: 0.959
Epoch: 2/10..  Training Loss: 0.176..  Test Loss: 0.139..  Test Accuracy: 0.961
Epoch: 3/10..  Training Loss: 0.160..  Test Loss: 0.132..  Test Accuracy: 0.963
Epoch: 4/10..  Training Loss: 0.147..  Test Loss: 0.126..  Test Accuracy: 0.964
Epoch: 5/10..  Training Loss: 0.137..  Test Loss: 0.119..  Test Accuracy: 0.965
Epoch: 6/10..  Training Loss: 0.128..  Test Loss: 0.117..  Test Accuracy: 0.967
Epoch: 7/10..  Training Loss: 0.118..  Test Loss: 0.111..  Test Accuracy: 0.969
Epoch: 8/10..  Training Loss: 0.113..  Test Loss: 0.108..  Test Accuracy: 0.971
Epoch: 9/10..  Training Loss: 0.104..  Test Loss: 0.108..  Test Accuracy: 0.970
Epoch: 10/10..  Training Loss: 0.094..  Test Loss: 0.106..  Test Accuracy: 0.971


With an accuracy of 0.971 we get number of parameters as 288560.

**2. Optimizer: Adam, learning rate = 0.001**

In [151]:
from torch import nn, optim
class Network(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(784, 50)
        self.fc2 = nn.Linear(50, 12)
        self.fc3 = nn.Linear(12, 24)
        self.fc4 = nn.Linear(24, 20)
        self.fc5 = nn.Linear(20, 10)

        # Dropout module with drop probability
        self.dropout = nn.Dropout(p=0.2)

    def forward(self, x):
        # make sure input tensor is flattened
        x = x.view(x.shape[0], -1)

        x = self.dropout(F.relu(self.fc1(x)))
        x = self.dropout(F.relu(self.fc2(x)))
        x = self.dropout(F.relu(self.fc3(x)))
        x = self.dropout(F.relu(self.fc4(x)))

        # No dropout at output
        x = F.log_softmax(self.fc5(x), dim=1)

        return x
        
model=Network()
print(model)
# specify loss function
criterion = nn.CrossEntropyLoss()

# specify optimizer
optimizer = optim.Adam(model.parameters(),lr=0.001)
                      

Network(
  (fc1): Linear(in_features=784, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=12, bias=True)
  (fc3): Linear(in_features=12, out_features=24, bias=True)
  (fc4): Linear(in_features=24, out_features=20, bias=True)
  (fc5): Linear(in_features=20, out_features=10, bias=True)
  (dropout): Dropout(p=0.2)
)


In [152]:
epochs=10
train_losses,test_losses=[],[]
for e in range(epochs):
    running_loss=0
    for images,labels in train_loader:
        optimizer.zero_grad()
        log_ps=model(images)
        loss=criterion(log_ps,labels)
        loss.backward()
        optimizer.step()
        running_loss+=loss.item()
        
    else:
        test_loss=0
        accuracy=0
        
        with torch.no_grad():
            model.eval()
            for images,labels in test_loader:
                log_ps=model(images)
                test_loss+=criterion(log_ps,labels)
                ps=torch.exp(log_ps)
                top_p,top_class=ps.topk(1,dim=1)
                equals=top_class==labels.view(*top_class.shape)
                accuracy+=torch.mean(equals.type(torch.FloatTensor))
        model.train()
        train_losses.append(running_loss/len(train_loader))
        test_losses.append(test_loss/len(test_loader))

        print("Epoch: {}/{}.. ".format(e+1, epochs),
              "Training Loss: {:.3f}.. ".format(running_loss/len(train_loader)),
              "Test Loss: {:.3f}.. ".format(test_loss/len(test_loader)),
              "Test Accuracy: {:.3f}".format(accuracy/len(test_loader)))


Epoch: 1/10..  Training Loss: 1.339..  Test Loss: 0.588..  Test Accuracy: 0.853
Epoch: 2/10..  Training Loss: 0.785..  Test Loss: 0.375..  Test Accuracy: 0.900
Epoch: 3/10..  Training Loss: 0.637..  Test Loss: 0.303..  Test Accuracy: 0.920
Epoch: 4/10..  Training Loss: 0.565..  Test Loss: 0.271..  Test Accuracy: 0.926
Epoch: 5/10..  Training Loss: 0.506..  Test Loss: 0.250..  Test Accuracy: 0.936
Epoch: 6/10..  Training Loss: 0.470..  Test Loss: 0.232..  Test Accuracy: 0.937
Epoch: 7/10..  Training Loss: 0.451..  Test Loss: 0.217..  Test Accuracy: 0.944
Epoch: 8/10..  Training Loss: 0.427..  Test Loss: 0.220..  Test Accuracy: 0.943
Epoch: 9/10..  Training Loss: 0.406..  Test Loss: 0.213..  Test Accuracy: 0.948
Epoch: 10/10..  Training Loss: 0.396..  Test Loss: 0.208..  Test Accuracy: 0.946


From above model we get an accuracy of 0.946 and number of parameters as 40768.